In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargar desde Google Drive
df_subset = pd.read_csv('/content/drive/My Drive/NLP/datos_procesados.csv')

 Creamos etiquetas binarias basadas en los ratings, dividimos los datos en características y etiquetas, y separamos esos datos en conjuntos de entrenamiento y prueba, asegurando un equilibrio entre las clases.

In [ ]:
# Crear la columna de etiquetas basada en el rating
df_subset['label'] = (df_subset['rating'] >= 4).astype(int)

# Dividir los datos en características (X) y etiquetas (y)
X = df_subset['processed_text']
y = df_subset['label']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")
print(f"Distribución de clases en el conjunto de entrenamiento:\n{y_train.value_counts(normalize=True)}")
print(f"Distribución de clases en el conjunto de prueba:\n{y_test.value_counts(normalize=True)}")

Tamaño del conjunto de entrenamiento: 6391
Tamaño del conjunto de prueba: 1598
Distribución de clases en el conjunto de entrenamiento:
label
0    0.500235
1    0.499765
Name: proportion, dtype: float64
Distribución de clases en el conjunto de prueba:
label
0    0.5
1    0.5
Name: proportion, dtype: float64


Realizaremos la vectorización del texto utilizando TF-IDF, limitando las características a las 8000 palabras más relevantes (reducir la dimensionalidad y prevenir sobreajuste), transformando los textos de entrenamiento y prueba en matrices numéricas adecuadas para el análisis. A continuación, inicializamos y entrenamos dos modelos de clasificación: Regresión Logística y Random Forest, utilizando los datos vectorizados. Luego, evaluamos el rendimiento de ambos modelos en el conjunto de prueba, generando informes de clasificación y matrices de confusión para cada uno. Finalmente, comparamos las predicciones de ambos modelos para determinar el grado de acuerdo entre ellos, proporcionando una visión sobre la consistencia y efectividad de las predicciones en la tarea de clasificación.

In [ ]:
# Vectorización del texto usando TF-IDF
vectorizer = TfidfVectorizer(max_features=8000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Inicializar y entrenar el modelo de Regresión Logística
logistic_model = LogisticRegression( max_iter=1000, random_state=42)
logistic_model.fit(X_train_vectorized, y_train)

# Inicializar y entrenar el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vectorized, y_train)

# Función para evaluar y mostrar resultados
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"\nResultados para {model_name}:")
    print(classification_report(y_test, y_pred))
    print("Matriz de Confusión:")
    print(confusion_matrix(y_test, y_pred))

# Evaluar ambos modelos
evaluate_model(logistic_model, X_test_vectorized, y_test, "Regresión Logística")
evaluate_model(rf_model, X_test_vectorized, y_test, "Random Forest")

# Comparar las predicciones de ambos modelos
y_pred_logistic = logistic_model.predict(X_test_vectorized)
y_pred_rf = rf_model.predict(X_test_vectorized)

print("\nComparación de predicciones:")
print(f"Acuerdo entre modelos: {np.mean(y_pred_logistic == y_pred_rf):.2%}")




Resultados para Regresión Logística:
              precision    recall  f1-score   support

           0       0.85      0.84      0.85       799
           1       0.84      0.85      0.85       799

    accuracy                           0.85      1598
   macro avg       0.85      0.85      0.85      1598
weighted avg       0.85      0.85      0.85      1598

Matriz de Confusión:
[[673 126]
 [119 680]]

Resultados para Random Forest:
              precision    recall  f1-score   support

           0       0.82      0.80      0.81       799
           1       0.81      0.82      0.81       799

    accuracy                           0.81      1598
   macro avg       0.81      0.81      0.81      1598
weighted avg       0.81      0.81      0.81      1598

Matriz de Confusión:
[[642 157]
 [142 657]]

Comparación de predicciones:
Acuerdo entre modelos: 89.61%


Los resultados de la evaluación de los modelos de Regresión Logística y Random Forest muestran que la Regresión Logística supera al Random Forest en precisión (0.85 frente a 0.81), recall y F1-score (aproximadamente 0.85 frente a 0.81), indicando un mejor rendimiento en la clasificación de sentimientos. La matriz de confusión revela que la Regresión Logística comete menos errores de clasificación, con un acuerdo del 89.61% entre las predicciones de ambos modelos, lo que sugiere que ambos capturan patrones similares en los datos, aunque la Regresión Logística es preferible para esta tarea específica.